In [316]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_transformer
from scipy.spatial.distance import cdist

Put all csv's in a list so they can later be merged.

In [317]:
partial_df_list = [
    pd.read_csv("data/icu_data_28.csv"),
    pd.read_csv("data/icu_data_33.csv"),
    pd.read_csv("data/icu_data_38_43.csv"),
    pd.read_csv("data/icu_data_48_53.csv"),
]

full_df_length = 0

for num, partial_df in enumerate(partial_df_list):
    print(f"The length of dataframe {num} is {len(partial_df)}")
    full_df_length += len(partial_df)

print("The total length of all the dataframes is:", full_df_length)

The length of dataframe 0 is 1746
The length of dataframe 1 is 1746
The length of dataframe 2 is 3492
The length of dataframe 3 is 3492
The total length of all the dataframes is: 10476


Create a dataframe with the partial dataframes.

In [318]:
full_df = pd.concat(partial_df_list)
full_df.head(5)

,Unnamed: 0,distribution,bed_amount,denied,covid_denied,waiting_time,occupancy,admissioned,failed_reschedules,succesful_reschedules
0,0,"[4, 4, 4, 9, 7]",28,1538,59,104382,63.485812,3213,8,969
1,1,"[4, 4, 4, 10, 6]",28,1623,74,102990,61.607143,3049,31,852
2,2,"[4, 4, 4, 11, 5]",28,1619,130,100212,62.344667,3140,23,875
3,3,"[4, 4, 4, 12, 4]",28,1638,207,84762,60.490460,2954,40,735
4,4,"[4, 4, 4, 13, 3]",28,1575,237,109320,57.014432,3052,32,858


The first column in the csv is the index column which is not relevant as we do not need to look at specific rows.

In [319]:
full_df.drop(columns="Unnamed: 0", inplace=True)
full_df.head()

,distribution,bed_amount,denied,covid_denied,waiting_time,occupancy,admissioned,failed_reschedules,succesful_reschedules
0,"[4, 4, 4, 9, 7]",28,1538,59,104382,63.485812,3213,8,969
1,"[4, 4, 4, 10, 6]",28,1623,74,102990,61.607143,3049,31,852
2,"[4, 4, 4, 11, 5]",28,1619,130,100212,62.344667,3140,23,875
3,"[4, 4, 4, 12, 4]",28,1638,207,84762,60.490460,2954,40,735
4,"[4, 4, 4, 13, 3]",28,1575,237,109320,57.014432,3052,32,858


Let's check if all rows are successfully merged.

In [320]:
print(f"The length of the new df is equal to the total amount of rows in all datasets: {len(full_df) == full_df_length}")

The length of the new df is equal to the total amount of rows in all datasets: True


There are some oddities in the dataset. First of all, the column `bed_amount` contains a list which a single value but 
the column should only contain integers so we should remove the brackets and turn the column into an integer. The 
brackets are removed using `replace()` and the very confusing regular expression `[\[\]]`. The latter consists out of
two parts: the enclosing brackets `[]` and what's within `\[\]`. The enclosing brackets means to select either one of 
the characters between them which we want to be `[` and `]` but they are already regex syntax so we have to escape them
using `\`.

In [321]:
full_df["bed_amount"].replace(r"[\[\]]", "", regex=True, inplace=True)
full_df["bed_amount"] = full_df["bed_amount"].astype(int)

print("The datatype of bed_amount is:", full_df["bed_amount"].dtype)

full_df["bed_amount"].head(5)

The datatype of bed_amount is: int64


0    28
1    28
2    28
3    28
4    28
Name: bed_amount, dtype: int64

Next is the `distribution` column which consists of a list with integers that correspond with the amount of beds 
dedicated to certain specialisms. The order is as following:

In [322]:
specs = ["CARD/INT/OTHER", "NEU/NEC", "CAPU", "CHIR", "COV"]
specs

['CARD/INT/OTHER', 'NEU/NEC', 'CAPU', 'CHIR', 'COV']

We can turn this column into the 5 columns. The column `distribution` is seen as a string so we need to turn it into a
list again. We do that by removing the brackets, splitting the string into a list using `split(",")` and then creating
5 new columns using these lists. After that the columns need to be converted from strings to ints.

In [323]:
full_df["distribution"].replace(r"[\[\]]", "", regex=True, inplace=True)
full_df[specs] = full_df["distribution"].apply(lambda x: x.split(",")[-5:]).tolist()
full_df[specs] = full_df[specs].astype(int)
full_df.drop(columns="distribution", inplace=True)

full_df.head()

,bed_amount,denied,covid_denied,waiting_time,occupancy,admissioned,failed_reschedules,succesful_reschedules,CARD/INT/OTHER,NEU/NEC,CAPU,CHIR,COV
0,28,1538,59,104382,63.485812,3213,8,969,4,4,4,9,7
1,28,1623,74,102990,61.607143,3049,31,852,4,4,4,10,6
2,28,1619,130,100212,62.344667,3140,23,875,4,4,4,11,5
3,28,1638,207,84762,60.490460,2954,40,735,4,4,4,12,4
4,28,1575,237,109320,57.014432,3052,32,858,4,4,4,13,3


We can now save this new dataframe into a csv file.

In [324]:
full_df.to_csv("data/full_monte_carlo.csv")
full_df[(full_df.succesful_reschedules == 208) & (full_df.admissioned == 5465)]

,bed_amount,denied,covid_denied,waiting_time,occupancy,admissioned,failed_reschedules,succesful_reschedules,CARD/INT/OTHER,NEU/NEC,CAPU,CHIR,COV


We do not care about occupancy below `75%` so we can discard those rows.

In [325]:
full_df = full_df[full_df["occupancy"] >= 75]
full_df.head()
print("New length of full_df:", len(full_df))

New length of full_df: 1085


This removes a large amount of rows. Let's focus on hte columns we want to min-max which are `denied`, 
`waiting_time`, `occupancy` and `failed_reschedules`.

In [326]:
min_max_columns = ["denied", "waiting_time", "occupancy", "failed_reschedules"]

Let's standardize these columns and display their distributions.

In [327]:
scaler = StandardScaler()
full_df[min_max_columns] = scaler.fit_transform(full_df[min_max_columns])
full_df.head()

,bed_amount,denied,covid_denied,waiting_time,occupancy,admissioned,failed_reschedules,succesful_reschedules,CARD/INT/OTHER,NEU/NEC,CAPU,CHIR,COV
175,28,0.914290,128,-0.347864,0.287250,3568,2.593752,302,4,6,8,5,5
211,28,1.299847,56,0.574936,-1.053612,3388,0.667523,770,4,7,5,6,6
218,28,0.980766,32,-0.343273,-0.997373,3513,0.956457,474,4,7,6,4,7
219,28,0.550893,78,-0.694402,-1.012172,3514,0.378588,373,4,7,6,5,6
227,28,0.453396,131,-0.574355,-1.031412,3550,2.690063,224,4,7,7,5,5


 Let's create a list with the ideal values, which is the 
lowest value for `denied`, `waiting_time` and `failed_reschedules` and the highest value for `occupancy`.

In [328]:
ideal_row = [full_df["denied"].min(), full_df["waiting_time"].min(), full_df["occupancy"].max(), full_df["failed_reschedules"].min()]
print(ideal_row)

[-2.396174818143296, -1.2621613523455562, 3.7563630483641397, -0.9697720739543891]


The ideal row is the theoretical best we can achieve with our dataset. We can now look through all the points and 
calculate the euclidean distance. The row with the smallest distance is the row that gets the closest to our theoretical
distance.

In [329]:
full_df["euclidean_distances"] = cdist(full_df[min_max_columns], np.array([ideal_row]))
full_df.head()

,bed_amount,denied,covid_denied,waiting_time,occupancy,admissioned,failed_reschedules,succesful_reschedules,CARD/INT/OTHER,NEU/NEC,CAPU,CHIR,COV,euclidean_distances
175,28,0.914290,128,-0.347864,0.287250,3568,2.593752,302,4,6,8,5,5,6.043887
211,28,1.299847,56,0.574936,-1.053612,3388,0.667523,770,4,7,5,6,6,6.546151
218,28,0.980766,32,-0.343273,-0.997373,3513,0.956457,474,4,7,6,4,7,6.209384
219,28,0.550893,78,-0.694402,-1.012172,3514,0.378588,373,4,7,6,5,6,5.793493
227,28,0.453396,131,-0.574355,-1.031412,3550,2.690063,224,4,7,7,5,5,6.701516


Now we can simply select the row with the lowest euclidean distance. The standardized columns will still be standardized
but this could be fixed by manually looking up the row corresponding to the other values.

In [330]:
full_df[full_df["euclidean_distances"] == full_df["euclidean_distances"].min()]

,bed_amount,denied,covid_denied,waiting_time,occupancy,admissioned,failed_reschedules,succesful_reschedules,CARD/INT/OTHER,NEU/NEC,CAPU,CHIR,COV,euclidean_distances
784,38,-0.362034,406,-0.722799,2.990824,4317,-0.777149,440,12,6,4,12,5,2.24762
